In [3]:
import numpy as np

# Basic 1-qubit basis states
zero = np.array([1,0], dtype=complex)
one  = np.array([0,1], dtype=complex)

def kron(*mats):
    """Tensor (Kronecker) product of many arrays."""
    out = np.array([1], dtype=complex)
    for M in mats:
        out = np.kron(out, M)
    return out

# ------------------------------
# 1) Tensor product of two vectors
# ------------------------------
v1 = (1/np.sqrt(2)) * (zero + one)      # |+>
v2 = np.array([1, 2], dtype=complex)    # arbitrary vector

tv = kron(v1, v2)
print("Tensor product v1 ⊗ v2 =\n", tv)
print()

# Normalize
tv_norm = tv / np.linalg.norm(tv)

# ------------------------------
# 2) Operator + Apply + Partial Measure
# ------------------------------
H = 1/np.sqrt(2) * np.array([[1,1],[1,-1]], dtype=complex)
X = np.array([[0,1],[1,0]], dtype=complex)

Op = np.kron(H, X)
state_before = tv_norm
state_after = Op.dot(state_before)

print("State after operator:\n", state_after)
print()

# Partial measurement of leftmost qubit
def partial_measure_leftmost(state, n_qubits):
    N = 2**n_qubits
    amps = state.reshape(N)
    probs = {}
    collapsed = {}

    for outcome in [0,1]:
        idxs = []
        for i in range(N):
            bits = format(i, f'0{n_qubits}b')
            if int(bits[0]) == outcome:
                idxs.append(i)
        idxs = np.array(idxs)
        sub = amps[idxs]
        p = np.sum(np.abs(sub)**2)
        probs[outcome] = p
        if p > 0:
            newstate = np.zeros_like(amps)
            newstate[idxs] = sub / np.sqrt(p)
            collapsed[outcome] = newstate
        else:
            collapsed[outcome] = None

    return probs, collapsed

probs, posts = partial_measure_leftmost(state_after, 2)
print("Probabilities:", probs)
print("Collapsed state if outcome 0:\n", posts[0])
print("Collapsed state if outcome 1:\n", posts[1])
print()

# ------------------------------
# 3) CSWAP (Fredkin gate)
# ------------------------------
dim = 8
CSWAP = np.zeros((dim,dim), dtype=complex)

def idx(bits):
    return int("".join(str(x) for x in bits), 2)

for i in range(dim):
    b = [int(x) for x in format(i,'03b')]   # (q0,q1,q2)
    if b[0] == 0:
        j = i
    else:
        newb = [b[0], b[2], b[1]]  # swap q1 and q2
        j = idx(newb)
    CSWAP[j,i] = 1

print("CSWAP matrix:\n", CSWAP)
print()

# Dirac form of CSWAP
SWAP = np.array([
    [1,0,0,0],
    [0,0,1,0],
    [0,1,0,0],
    [0,0,0,1]
], dtype=complex)

proj0 = np.outer(zero, zero)
proj1 = np.outer(one, one)

CSWAP_expr = np.kron(proj0, np.eye(4)) + np.kron(proj1, SWAP)
print("CSWAP correct via projector method? ", np.allclose(CSWAP, CSWAP_expr))
print()
print("Dirac form:")
print("CSWAP = |0⟩⟨0| ⊗ I ⊗ I  +  |1⟩⟨1| ⊗ SWAP")
print()

# ------------------------------
# 4) Toffoli (CCX)
# ------------------------------
TOF = np.eye(8, dtype=complex)
for i in range(8):
    b = [int(x) for x in format(i,'03b')]
    if b[0] == 1 and b[1] == 1:  # controls
        newb = [b[0], b[1], 1 - b[2]]
        j = idx(newb)
        TOF[j,i] = 1
        TOF[i,i] = 0

print("Toffoli matrix:\n", TOF)
print()

# Dirac notation
# Projector |11><11| on the control qubits
basis11 = np.array([0,0,0,1], dtype=complex)  # |11>
proj_11 = np.outer(basis11, basis11)          # 4×4 control projector
X_t = np.array([[0,1],[1,0]], dtype=complex)

TOF_expr = np.kron(np.eye(4) - proj_11, np.eye(2)) + np.kron(proj_11, X_t)
print("Toffoli correct via projector method? ", np.allclose(TOF, TOF_expr))
print()
print("Dirac form:")
print("TOFFOLI = (I - |11⟩⟨11|) ⊗ I  +  |11⟩⟨11| ⊗ X")
print()

# Example action
v = np.zeros(8, dtype=complex); v[idx([1,1,0])] = 1
print("Toffoli acting on |110> =\n", TOF.dot(v))


Tensor product v1 ⊗ v2 =
 [0.70710678+0.j 1.41421356+0.j 0.70710678+0.j 1.41421356+0.j]

State after operator:
 [8.94427191e-01+0.j 4.47213595e-01+0.j 1.67871766e-17+0.j
 8.39358832e-18+0.j]

Probabilities: {0: np.float64(0.9999999999999999), 1: np.float64(3.522616244659067e-34)}
Collapsed state if outcome 0:
 [0.89442719+0.j 0.4472136 +0.j 0.        +0.j 0.        +0.j]
Collapsed state if outcome 1:
 [0.        +0.j 0.        +0.j 0.89442719+0.j 0.4472136 +0.j]

CSWAP matrix:
 [[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

CSWAP correct via projector method?  True



In [2]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator, partial_trace, DensityMatrix
import numpy as np

# --------------------------------------------------------------------
# 1. Tensor product (Kronecker product) of two vectors
# --------------------------------------------------------------------

v1 = np.array([1/np.sqrt(2), 1/np.sqrt(2)])     # |+>
v2 = np.array([1, 2], dtype=complex)            # arbitrary vector

tensor_product = np.kron(v1, v2)
print("Tensor Product = \n", tensor_product)


# --------------------------------------------------------------------
# 2. Create an operator, apply it on a statevector, and partial measure
# --------------------------------------------------------------------

# 2-qubit initial state |ψ0>
psi0 = Statevector.from_label("00")

# Create operator: H on qubit 0, X on qubit 1
qc_op = QuantumCircuit(2)
qc_op.h(0)
qc_op.x(1)
Op = Operator(qc_op)

# Apply operator on statevector
psi1 = psi0.evolve(Op)
print("\nState After Operator:\n", psi1)

# Partial measurement on qubit 0
rho = DensityMatrix(psi1)
reduced = partial_trace(rho, [1])  # trace out qubit 1 → measure qubit 0
print("\nReduced Density Matrix (for qubit 0):\n", reduced.data)

p0 = rho.probabilities([0])[0]
p1 = rho.probabilities([0])[1]
print("\nMeasurement Probabilities of qubit 0: P(0) =", p0, ", P(1) =", p1)


# --------------------------------------------------------------------
# 3. CSWAP (Fredkin) Matrix Representation
# --------------------------------------------------------------------

qc_cswap = QuantumCircuit(3)
qc_cswap.cswap(0, 1, 2)

CSWAP = Operator(qc_cswap)
print("\nCSWAP Matrix =\n", CSWAP.data)

print("\nDirac Notation for CSWAP:")
print("CSWAP = |0><0| ⊗ I ⊗ I  +  |1><1| ⊗ SWAP")
print("where SWAP = Σ |ij⟩⟨ji| over i,j ∈ {0,1}")


# --------------------------------------------------------------------
# 4. Toffoli (CCX) Matrix Representation
# --------------------------------------------------------------------

qc_toff = QuantumCircuit(3)
qc_toff.ccx(0, 1, 2)

TOFFOLI = Operator(qc_toff)
print("\nToffoli Matrix =\n", TOFFOLI.data)

print("\nDirac Notation for Toffoli:")
print("TOFFOLI = (I - |11><11|) ⊗ I  +  |11><11| ⊗ X")
print("Action: |a b c⟩ → |a b, c XOR (a AND b)⟩")


Tensor Product = 
 [0.70710678+0.j 1.41421356+0.j 0.70710678+0.j 1.41421356+0.j]

State After Operator:
 Statevector([0.        +0.j, 0.        +0.j, 0.70710678+0.j,
             0.70710678+0.j],
            dims=(2, 2))

Reduced Density Matrix (for qubit 0):
 [[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]

Measurement Probabilities of qubit 0: P(0) = 0.4999999999999999 , P(1) = 0.4999999999999999

CSWAP Matrix =
 [[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

Dirac Notation for CSWAP:
CSWAP = |0><0| ⊗ I ⊗ I  +  |1><1| ⊗ SWAP
where SWAP = Σ |ij⟩⟨ji| over i,j ∈ {0,1}

Toffoli